<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pickle
# pd.options.display.max_rows = 300
import geojson
from transliterate import translit
from similar_text import similar_text
import plotly.express as px
from jupyter_dash import JupyterDash
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_vtk
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash import callback_context
import plotly.express as px
import numpy as np
import geojson
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import heapq
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

D:\anaconda\lib\site-packages\ipykernel_launcher.py:22: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
D:\anaconda\lib\site-packages\ipykernel_launcher.py:24: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`


In [17]:
df = pd.read_csv('train.csv')
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [18]:
df["build_year"] = df["build_year"].replace([0, 1, 20052009, 3, 20, 4965, 215, 71], [2000, 2001, 2009, 2003, 2020, 1965, 2015, 1971])

In [19]:
replace_index = df[df.life_sq > df.full_sq].index
df.loc[replace_index, "life_sq"] = np.NaN
replace_index = df[df.kitch_sq  > df.full_sq].index
df.loc[replace_index, "kitch_sq"] = np.NaN
replace_index = df[(df.kitch_sq == 0).values + (df.kitch_sq == 1).values].index
df.loc[replace_index, "kitch_sq"] = np.NaN

df = df[df.price_doc/df.full_sq <= 600000]
df = df[df.price_doc/df.full_sq >= 40000]

In [20]:
target_col=df['price_doc'].values
df.drop('price_doc', axis=1,inplace=True)

In [21]:
df.drop(['id','timestamp'], axis=1, inplace=True)

In [22]:
feature_forms = ['build_year', 'full_sq', 'kitch_sq', 'life_sq', 'num_room', 'floor', 'max_floor']

In [23]:
df_all = df[feature_forms]

In [24]:
for c in df_all.columns:
    if df_all[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(df_all[c].values))
        df_all[c] = lbl.transform(list(df_all[c].values))

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_all, 
                                                    target_col, 
                                                    test_size=0.2, 
                                                    random_state=31,
                                                   shuffle=False)

In [26]:
xgb_train = xgb.DMatrix(X_train, np.log1p(y_train), feature_names=X_train.columns.values)
xgb_valid = xgb.DMatrix(X_test, feature_names=X_test.columns.values)

In [27]:
xgb_params = {
    'n_estimators' : 300,
    'eta': 0.05,
    'max_depth': 5,
    'objective': 'reg:linear',
    'eval_metric' : 'rmse',
    'silent': 1
}

In [28]:
model = xgb.train(xgb_params, xgb_train, num_boost_round=100)
pred = np.exp(model.predict(xgb_valid))

[16:23:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:23:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [29]:
rmsle = np.sqrt(mean_squared_log_error(y_test, pred))

print('RMSLE: {:.3f}'.format(rmsle))

RMSLE: 0.356


In [15]:
with open(r'xgb_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [16]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Machine Learning Model Deployment'
server = app.server

with open('xgb_model.pickle', 'rb') as f:
    clf = pickle.load(f)


app.layout = html.Div([
    dbc.Row([html.H3(children='Price predict')]),
    html.H6(["Год постройки: ", dcc.Input(id='build_year', value='', type='text', style={'width': 80})
    ]),
    html.H6(["Общая площадь: ", dcc.Input(id='full_sq', value='', type='text', style={'width': 80})
    ]),
    html.H6(["Площадь кухни: ", dcc.Input(id='kitch_sq', value='', type='text', style={'width': 80})
    ]),
    html.H6(["Жилая площадь: ", dcc.Input(id='life_sq', value='', type='text', style={'width': 80})
    ]),
    html.H6(["Этажность дома: ", dcc.Input(id='max_floor', value='', type='text', style={'width': 80})
    ]),
    html.H6(["Этаж квартиры: ", dcc.Input(id='floor', value='', type='text', style={'width': 80})
    ]),
    html.H6(["Кол-во комнат: ", dcc.Input(id='num_room', value='', type='text', style={'width': 80})
    ]),
    html.Br(),
    dbc.Row([html.Div(id='prediction output')])
]) 

@app.callback(
    Output('prediction output', 'children'),
    Input(component_id='build_year', component_property='value'),
    Input(component_id='full_sq', component_property='value'),
    Input(component_id='kitch_sq', component_property='value'),
    Input(component_id='life_sq', component_property='value'),
    Input(component_id='floor', component_property='value'),
    Input(component_id='max_floor', component_property='value'),
    Input(component_id='num_room', component_property='value')
)
   
def update_output( build_year, full_sq, kitch_sq, floor, max_floor, life_sq, num_room):    
    x = np.array([[float(build_year), float(full_sq), float(kitch_sq), float(floor), float(max_floor), float(life_sq), float(num_room)]])
    data_for_predict = pd.DataFrame(x, columns = ['build_year', 'full_sq', 'kitch_sq', 'life_sq', 'num_room', 'floor', 'max_floor'])
    res = np.exp(clf.predict(xgb.DMatrix(data_for_predict, feature_names=data_for_predict.columns.values)))
    return f'The predicted price is {res[0]}'

                   
if __name__ == '__main__':
    app.run_server(debug=False,port=7010)

Dash is running on http://127.0.0.1:7010/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:7010/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_0m1658325669.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_0m1658325669.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_0m1658325669.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET /_dash-component-suites/dash/deps/react@16.v2_6_0m1658325669.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_0m1658765824.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET /_dash-component-suites/dash_vtk/dash_vtk.v0_0_9m1659193050.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:15] "GET /_dash-c

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\anaconda\lib\site-packages\dash\dash.py", line 1266, in dispatch
    callback_context=g,
  File "D:\anaconda\lib\site-packages\dash\_callback.py", line 439, in add_context
  

127.0.0.1 - - [29/Aug/2022 16:03:16] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\anaconda\lib\site-packages\dash\dash.py", line 1266, in dispatch
    callback_context=g,
  File "D:\anaconda\lib\site-packages\dash\_callback.py", line 439, in add_context
  

...

127.0.0.1 - - [29/Aug/2022 16:03:31] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\anaconda\lib\site-packages\dash\dash.py", line 1266, in dispatch
    callback_context=g,
  File "D:\anaconda\lib\site-packages\dash\_callback.py", line 439, in add_context
  

127.0.0.1 - - [29/Aug/2022 16:03:31] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\anaconda\lib\site-packages\dash\dash.py", line 1266, in dispatch
    callback_context=g,
  File "D:\anaconda\lib\site-packages\dash\_callback.py", line 439, in add_context
  

127.0.0.1 - - [29/Aug/2022 16:03:32] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [29/Aug/2022 16:03:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:03:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:04:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:04:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2022 16:04:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.